In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'no title found'

        # Remove irrelevant tags
        for tag in soup.body(['script', 'style', 'img', 'input']):
            tag.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)


In [3]:
MODEL = "llama3.2"

In [4]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt(Website):
    prompt = f"""You are looking at a website titled: {Website.title}
    The content of the website is as follows: {Website.text}
    Please provide a short summary of this website in markdown. 
    If it includes news or announcements, then summarize them too. 
"""
    return prompt


In [ ]:
def messages(Website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(Website)}
    ]

In [10]:
def summarize(url) :
    website = Website(url)
    response = ollama.chat(MODEL,messages(website))
    print(response['message']['content'])

In [11]:
website_summary = summarize("https://edwarddonner.com")

**Website Summary**
```markdown
# Edward Donner's Website

## About the Creator

* Ed is the co-founder and CTO of Nebula.io, applying AI to help people discover their potential.
* He's previously founded and CEO'd AI startup untapt, acquired in 2021.

## Recent Announcements

* **The Complete Agentic AI Engineering Course** (April 21, 2025)
* **LLM Workshop – Hands-on with Agents – resources** (December 21, 2024)
* **Mastering AI and LLM Engineering – Resources** (November 13, 2024)

## Projects

* Connect Four: a game featuring artificial intelligence.
* Outsmart: an arena pitting LLMs against each other in a battle of diplomacy and deviousness.
```
Note that I've ignored navigation-related text and only summarized the content, announcements, and projects.
